In [1]:
import pandas as pd
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import tensorflow as tf
import sklearn
from tqdm import tqdm
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Import the questions and accepted answers

In [5]:
accepted_answers_df = pd.read_csv('Accepted_Answers.csv')
accepted_answers_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
9995,"Spring Security Core authAjax, How do I ignore...",<p>When you make an un-authorized attempt to a...,5628199
9996,Dictionary+Queue Data Structure with Active Re...,<p>I'd start by just modeling the behavior you...,5888734
9997,Extend an existing list in Django settings,<p>You should be able to import form <code>dja...,5916209
9998,Python: How to deal with non-subscriptable obj...,<p>The way to fix it is to use dictionaries pr...,5977165


Import the questions and highest scored answers

In [6]:
scored_answer_df = pd.read_csv('Scored_Answers.csv')
scored_answer_df

,Question,Relevant_Answer,question_id
0,in android how to send and receive special sms...,<p>For this you have to create your custom bro...,29165268
1,Python Salary Program Using Conditionals,<p>You're using full quotes where you should b...,29089187
2,How can I show a spinner in a ViewPager when t...,<p>I would do a check for this inside the <cod...,29169909
3,Android: Horizontal list view on custom adaptor,<blockquote>\n <p>layout whose one item cover...,28878917
4,"Even If Testcase failed It is going in ""onTest...",<p>Function <code>verify</code> won't make a t...,29300800
...,...,...,...
9995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
9996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
9997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
9998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


Combine the two dataframes to create dataframe with questions and their relevant answer

In [7]:
ground_truth_df = pd.concat([accepted_answers_df, scored_answer_df], axis=0, ignore_index=True)
ground_truth_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


Get all answers data for training

In [8]:
all_answers_df = pd.read_csv('All_answers.csv')
all_answers_df

,question_id,question_title,answer_id,answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...
...,...,...,...,...
33424,73829515,Big Query first non null value record type,73830169,<p>Consider below option</p>\n<pre><code>selec...
33425,73829809,Change Wordpress colors based on logo main color,73829900,"<p>First of all, you need colors from the imag..."
33426,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...
33427,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...


## Functions to clean the answers columns

In [9]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r' ', text)

def remove_punctuation_and_lower(text):
    text = text.lower()
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word not in stop_words]

def remove_usernames(text: str) -> str:
    """
    Remove usernames starting with '@' from the given text.

    Args:
        text (str): The input text containing usernames.

    Returns:
        str: The text with usernames removed.
    """
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", " ", text)
    return cleaned_text

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_usernames(text)
    text = remove_urls(text)
    text = remove_punctuation_and_lower(text)
    text_without_newlines = text.replace('\n', ' ')
    # words = remove_stopwords(text)
    return text_without_newlines


In [10]:
ground_truth_df['Clean_Relevant_Answer'] = ground_truth_df['Relevant_Answer'].apply(clean_text)
ground_truth_df

,Question,Relevant_Answer,question_id,Clean_Relevant_Answer
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519,you dont appear to be returning anything from ...
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802,perl to the rescue perl e print packb argv0 0...
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188,your variables are mutated from inside of an a...
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556,attributestokenidlastclaimdate is a uint so at...
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861,conn is not defined here either change connect...
...,...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549,i am using the version 0616 you can set the ...
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033,your question doesnt really have anything to d...
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186,using a break instruction is a possibility but...
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275,you can try using f800 for englishus locale as...


In [11]:
all_answers_df['clean_answer_body'] = all_answers_df['answer_body'].apply(clean_text)
all_answers_df.head()

,question_id,question_title,answer_id,answer_body,clean_answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...,thats excellent for something written from ...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...,if you are intending on reading a large number...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...,if you are querying a sql server database vers...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...,very roughly and from memory since i dont have...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...,thats definitely a good way to do it but you ...


In [12]:
# Group the rows by question_id and aggregate the other columns into lists
grouped_answers_df = all_answers_df.groupby('question_id').agg({
    'question_title': lambda x: list(x)[0],
    'answer_id': lambda x: list(x),
    'answer_body': lambda x: list(x),
    'clean_answer_body': lambda x: list(x)
})

In [13]:
# Reset the index and rename the columns
grouped_answers_df = grouped_answers_df.reset_index().rename(columns={
    'answer_id': 'answer_ids',
    'answer_body': 'answer_bodies',
    'clean_answer_body': 'clean_answer_bodies'
})
grouped_answers_df

,question_id,question_title,answer_ids,answer_bodies,clean_answer_bodies
0,930,How do I connect to a database and loop over a...,"[951, 65774, 20049, 940, 1022, 2369, 56179247,...",[<p>@Goyuix -- that's excellent for something ...,[ thats excellent for something written from...
1,22979,Detecting if SQL server is running,"[451160, 23019, 23011, 23010, 22991, 4217054]",[<p><code>System.Data.Sql.SqlDataSourceEnumera...,[systemdatasqlsqldatasourceenumerator will ret...
2,23027,CruiseControl.Net Build Publisher - Only publi...,"[32667, 32681, 23060, 7028931]",[<p>The default build publisher in CC.NET does...,[the default build publisher in ccnet does not...
3,34896,When is it best to sanitize user input?,"[34906, 34905, 34984, 34902, 36088, 34923, 349...",[<p>Users are evil!</p>\n\n<p>Well perhaps not...,[users are evil well perhaps not always but my...
4,37263,"Where does ""Change Management"" end and ""Projec...","[43187, 37565, 37277, 37291, 37292]","[<p><strong>What is change management, and how...",[what is change management and how does it app...
...,...,...,...,...,...
19995,73829515,Big Query first non null value record type,"[73829947, 73830169]",[<p>You may consider below scalar subquery for...,[you may consider below scalar subquery for yo...
19996,73829809,Change Wordpress colors based on logo main color,[73829900],"[<p>First of all, you need colors from the ima...",[first of all you need colors from the image y...
19997,73831805,Why are methods not valid constructors?,[73832791],[<p>As pointed out in the comments on the orig...,[as pointed out in the comments on the origina...
19998,73834279,No ability to publish newer build for external...,[73834304],[<p>I believe that there is possible to do mor...,[i believe that there is possible to do more e...


In [14]:
final_df = pd.merge(grouped_answers_df, ground_truth_df, on='question_id')
final_df.head()

,question_id,question_title,answer_ids,answer_bodies,clean_answer_bodies,Question,Relevant_Answer,Clean_Relevant_Answer
0,89226,Is there a tool to convert a .vim colour defin...,[89300],[<p>Here is a Visual Studio theme generator wh...,[here is a visual studio theme generator which...,Is there a tool to convert a .vim colour defin...,<p>Here is a Visual Studio theme generator whi...,here is a visual studio theme generator which ...
1,119792,"Subversion Berkeley DB broken, recovery failed","[119798, 119957, 119862, 2851717]",[<blockquote>\n <p>I've got a Subversion repo...,[ ive got a subversion repository backed by th...,"Subversion Berkeley DB broken, recovery failed",<blockquote>\n <p>I've got a Subversion repos...,ive got a subversion repository backed by the...
2,156089,Silverlight 2.0 DataGrid How to remove mouseov...,"[183936, 1681462]",[<p>Yeah You have to change the Style and Cont...,[yeah you have to change the style and control...,Silverlight 2.0 DataGrid How to remove mouseov...,<p>The short answer is to use styles. The lon...,the short answer is to use styles the long an...
3,181241,"Why does the ""at"" command always warn me that ...","[182525, 181245, 181252, 55071622]",[<p>Does the warning have any harmful effect a...,[does the warning have any harmful effect asid...,"Why does the ""at"" command always warn me that ...",<p>It serves as a good warning to those of us ...,it serves as a good warning to those of us tha...
4,236359,What is your convention to distinguish between...,"[237976, 236596, 236402]",[<p>I'd like to make the suggestion that when ...,[id like to make the suggestion that when you ...,What is your convention to distinguish between...,<pre><code>use no underscores for the external...,use no underscores for the external api one un...


### Get the data of the questions and answers in required format

In [39]:
## Using unprocessed data to create question and answer pairs

from sklearn.model_selection import train_test_split

# Create question-answer pairs and labels
input_pairs = []
labels = []

questions_list = final_df['question_title'].values
answers_list = final_df['clean_answer_bodies'].values
correct_answers_list = final_df['Clean_Relevant_Answer'].values

for question, ans_list in zip(questions_list, answers_list):
    for answer in ans_list:
        input_pairs.append((question, answer))
        labels.append(1 if answer in correct_answers_list else 0)


In [40]:
# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(input_pairs, labels, test_size=0.2, random_state=42, stratify=labels)

## ELmo model

Preprocessing for model

In [41]:
train_x = np.array([' '.join(t) for t in train_data]).reshape(len(train_data),1)
val_x = np.array([' '.join(t) for t in val_data]).reshape(len(val_data),1)

train_y = np.array(train_labels).reshape(len(train_labels),1)
val_y = np.array(val_labels).reshape(len(val_labels),1)

In [42]:
del input_pairs
del labels
del train_data
del val_data

Model architecture for fine-tuning

In [43]:
tf.compat.v1.disable_eager_execution()

In [44]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, Layer
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [22]:
class ElmoEmbeddingLayer(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        self.dimensions=1024
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=self.trainable, name=f'{self.name}_module')
        if self.trainable:
            self._trainable_weights.extend(tf.compat.v1.trainable_variables(scope=f'^{self.name}_module/.*'))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, dtype=tf.string), axis=1), as_dict=True, signature='default')['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [23]:
def create_model(train_elmo=False):
  ## input layer and embedding for questions
  input_text = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
  embedding = ElmoEmbeddingLayer(trainable=train_elmo)(input_text)

  ## output layer
  output = tf.keras.layers.Dense(1, activation='sigmoid')(embedding)

  model = tf.keras.Model(inputs=input_text, outputs=output)

  sess = tf.compat.v1.keras.backend.get_session()
  init = tf.compat.v1.global_variables_initializer()
  sess.run(init)

  optimizer_fn = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)
  model.compile(optimizer=optimizer_fn, loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [24]:
## Finetuning
finetune_elmo = create_model(train_elmo=True)
finetune_elmo.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 elmo_embedding_layer (Elmo  (None, 1024)              4         
 EmbeddingLayer)                                                 
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 1029 (4.02 KB)
Trainable params: 1029 (4.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
### creating a folder in current directory for saving model weights

folder_path ='./ELMo_model'

if os.path.exists(folder_path) == True:
    None
else:
    os.makedirs(folder_path)

In [ ]:
# ## saving weights

checkpoint_path = './ELMo_model/weight_{epoch:02d}.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='max',
                                                monitor='accuracy')
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=5,restore_best_weights=True)

checkpt_list = [checkpoint, early]

Training

In [26]:
EPOCHS = 5
BATCH_SIZE = 10
STEPS_PER_EPOCH = len(train_labels)//BATCH_SIZE
VAL_STEPS = len(val_labels)//BATCH_SIZE

In [45]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(len(train_x)).batch(BATCH_SIZE)
val_dataset = train_dataset.shuffle(len(val_x)).batch(BATCH_SIZE)

In [ ]:
finetune_elmo.fit(train_dataset, epochs=5, callbacks=checkpt_list)

Train on 371 steps
Epoch 1/5
371/371 [==============================] - 293s 789ms/step - batch: 185.0000 - size: 1.0000 - loss: 0.6363 - accuracy: 0.6475
Epoch 2/5
371/371 [==============================] - 281s 757ms/step - batch: 185.0000 - size: 1.0000 - loss: 0.6316 - accuracy: 0.6561
Epoch 3/5
371/371 [==============================] - 280s 754ms/step - batch: 185.0000 - size: 1.0000 - loss: 0.6269 - accuracy: 0.6561
Epoch 4/5
371/371 [==============================] - 279s 753ms/step - batch: 185.0000 - size: 1.0000 - loss: 0.6223 - accuracy: 0.6594
Epoch 5/5
371/371 [==============================] - 279s 752ms/step - batch: 185.0000 - size: 1.0000 - loss: 0.6202 - accuracy: 0.6640


In [ ]:
finetune_elmo.fit(train_dataset, epochs=5, callbacks=checkpt_list)

Train on 578 steps
Epoch 1/5
578/578 [==============================] - 477s 824ms/step - batch: 288.5000 - size: 1.0000 - loss: 0.5890 - accuracy: 0.7029
Epoch 2/5
578/578 [==============================] - 478s 827ms/step - batch: 288.5000 - size: 1.0000 - loss: 0.5877 - accuracy: 0.7024
Epoch 3/5
578/578 [==============================] - 479s 829ms/step - batch: 288.5000 - size: 1.0000 - loss: 0.5871 - accuracy: 0.7002
Epoch 4/5
578/578 [==============================] - 483s 836ms/step - batch: 288.5000 - size: 1.0000 - loss: 0.5836 - accuracy: 0.7078
Epoch 5/5
578/578 [==============================] - 481s 832ms/step - batch: 288.5000 - size: 1.0000 - loss: 0.5800 - accuracy: 0.7036


In [46]:
## run this
finetune_elmo.load_weights('./ELMo_model/weight_05.hdf5')

Testing the model

In [ ]:
# Get the input layer and encoder layer from the trained model
input_layer = finetune_elmo.input
embb_layer = finetune_elmo.layers[1].output

# Create a new model that takes the input layer and encoder layer as inputs and outputs the encoder layer's output
elmo_embb_model = tf.keras.Model(inputs=input_layer, outputs=embb_layer)

In [ ]:
elmo_embb_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 elmo_embedding_layer (Elmo  (None, 1024)              4         
 EmbeddingLayer)                                                 
                                                                 
Total params: 4 (16.00 Byte)
Trainable params: 4 (16.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
sample_df = final_df[:2000]
sample_df.head()

,question_id,question_title,answer_ids,answer_bodies,clean_answer_bodies,Question,Relevant_Answer,Clean_Relevant_Answer
0,89226,Is there a tool to convert a .vim colour defin...,[89300],[<p>Here is a Visual Studio theme generator wh...,[here is a visual studio theme generator which...,Is there a tool to convert a .vim colour defin...,<p>Here is a Visual Studio theme generator whi...,here is a visual studio theme generator which ...
1,119792,"Subversion Berkeley DB broken, recovery failed","[119798, 119957, 119862, 2851717]",[<blockquote>\n <p>I've got a Subversion repo...,[ ive got a subversion repository backed by th...,"Subversion Berkeley DB broken, recovery failed",<blockquote>\n <p>I've got a Subversion repos...,ive got a subversion repository backed by the...
2,156089,Silverlight 2.0 DataGrid How to remove mouseov...,"[183936, 1681462]",[<p>Yeah You have to change the Style and Cont...,[yeah you have to change the style and control...,Silverlight 2.0 DataGrid How to remove mouseov...,<p>The short answer is to use styles. The lon...,the short answer is to use styles the long an...
3,181241,"Why does the ""at"" command always warn me that ...","[182525, 181245, 181252, 55071622]",[<p>Does the warning have any harmful effect a...,[does the warning have any harmful effect asid...,"Why does the ""at"" command always warn me that ...",<p>It serves as a good warning to those of us ...,it serves as a good warning to those of us tha...
4,236359,What is your convention to distinguish between...,"[237976, 236596, 236402]",[<p>I'd like to make the suggestion that when ...,[id like to make the suggestion that when you ...,What is your convention to distinguish between...,<pre><code>use no underscores for the external...,use no underscores for the external api one un...


In [ ]:
questions = sample_df['Question'].values

In [ ]:
questions = questions.reshape(len(questions),1)

In [ ]:
questions.shape

(2000, 1)

In [ ]:
questions_embeddings = [elmo_embb_model.predict(q) for q in questions]

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
np.save('finetuned_elmo_question_embeddings.npy', questions_embeddings)

In [ ]:
!pip install faiss-cpu  # for CPU version
# or
!pip install faiss-gpu  # for GPU version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.2 MB/s eta 0:00:00


In [ ]:
import faiss

In [ ]:
# Convert question embeddings to a numpy matrix
question_embeddings_matrix = np.vstack(questions_embeddings)

# Get the dimension of the embeddings
dimension = question_embeddings_matrix.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# Add the question embeddings to the index
index.add(question_embeddings_matrix)

In [ ]:
def find_top_similar_question_indices(user_input, model, index, k=5):
    # Encode the user input
    embedding = model.predict([user_input])

    # Search the index for the top k similar embeddings
    _, indices = index.search(embedding, k)

    # # Flatten the indices array
    indices = indices.flatten()

    return indices

In [ ]:
sample_df.columns

Index(['question_id', 'question_title', 'answer_ids', 'answer_bodies',
       'clean_answer_bodies', 'Question', 'Relevant_Answer',
       'Clean_Relevant_Answer'],
      dtype='object')

In [ ]:
# user_input = "What is the procedure for establishing a database connection and iterating over a recordset in C#?"
user_question = "How to use regular expression in python?"

user_question = np.array([user_question])
top_question_indices = find_top_similar_question_indices(user_question, elmo_embb_model, index)

print("Indices of the top 5 similar questions:", top_question_indices)

Indices of the top 5 similar questions: [95 43 28 24 49]


In [ ]:
top_answers = sample_df.loc[top_question_indices, 'Relevant_Answer']

print("Top 5 answers:")
print()
i = 1
for answer in top_answers:
    print(i)
    print(f"{remove_html_tags(answer)}")
    i += 1
    print('**********************************************************************************************')

Top 5 answers:

1
Using regular expressions to pull values from HTML is always a mistake.  HTML syntax is a lot more complex that it may first appear and it's very easy for a page to catch out even a very complex regular expression.
Use an HTML Parser instead. See also What are the pros and cons of the leading Java HTML parsers?
**********************************************************************************************
2
\$bar->{baz}

should work.
E.g.:
my $foo;
$foo->{bar} = 123;

my $bar = \$foo->{bar};

$$bar = 456;

print "$foo->{bar}\n";   # prints "456"

In answer to the update in the OP, you can do:
\@$row{qw(body data)};

This is not the same as \@array, which would create one reference to an array. The above will distribute the reference and make a list of two references.
**********************************************************************************************
3
Just to clarify: the code in the body of a module is not executed every time the module is imported - it is 